In [ ]:
!pip install cfbd

In [5]:
import numpy as np
import pandas as pd
import requests
import cfbd
from cfbd.rest import ApiException

In [6]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'qceWlMfANbFaT2+498WLmV0a3YoSPAiEODUnOls/vtfAFLQ+MI9COwwcP/iGhIx7'
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [15]:
api_instance = cfbd.PlaysApi(cfbd.ApiClient(configuration))
# List to store the lines data for each year
all_plays = []

for year in range(2015, 2023):
  for week in range(1,15):
    try:
    # Play by play data
      api_response = api_instance.get_plays(year=year, week=week, team='Clemson')
      all_plays.extend(api_response)
    except ApiException as e:
      print("Exception when calling PlaysApi->get_plays: %s\n" % e)

In [23]:
df = pd.DataFrame([
    dict(
        home = l.home,
        away = l.away,
        offense_score = l.offense_score,
        defense_score = l.defense_score,
        period = l.period,
        clock_minutes = l.clock.minutes,
        clock_seconds = l.clock.seconds,
        yards_to_goal = l.yards_to_goal,
        down = l.down,
        distance = l.distance,
        play_type = l.play_type
        )
    for l in all_plays])
df.head()

,home,away,offense_score,defense_score,period,clock_minutes,clock_seconds,yards_to_goal,down,distance,play_type
0,Clemson,Wofford,0,0,1,15,0,65,1,10,Kickoff
1,Clemson,Wofford,0,0,1,15,0,75,1,10,Rush
2,Clemson,Wofford,0,0,1,14,40,71,2,6,Rush
3,Clemson,Wofford,0,0,1,14,15,66,3,1,Rush
4,Clemson,Wofford,0,0,1,13,45,64,1,10,Penalty


In [26]:
df['is_home'] = np.where(df['home'] == 'Clemson', 1, 0)
df.head()

,home,away,offense_score,defense_score,period,clock_minutes,clock_seconds,yards_to_goal,down,distance,play_type,is_home
0,Clemson,Wofford,0,0,1,15,0,65,1,10,Kickoff,1
1,Clemson,Wofford,0,0,1,15,0,75,1,10,Rush,1
2,Clemson,Wofford,0,0,1,14,40,71,2,6,Rush,1
3,Clemson,Wofford,0,0,1,14,15,66,3,1,Rush,1
4,Clemson,Wofford,0,0,1,13,45,64,1,10,Penalty,1


In [28]:
df['seconds_remaining'] = (df['clock_minutes'] * 60) + df['clock_seconds']
df.head()

,home,away,offense_score,defense_score,period,clock_minutes,clock_seconds,yards_to_goal,down,distance,play_type,is_home,seconds_remaining
0,Clemson,Wofford,0,0,1,15,0,65,1,10,Kickoff,1,900
1,Clemson,Wofford,0,0,1,15,0,75,1,10,Rush,1,900
2,Clemson,Wofford,0,0,1,14,40,71,2,6,Rush,1,880
3,Clemson,Wofford,0,0,1,14,15,66,3,1,Rush,1,855
4,Clemson,Wofford,0,0,1,13,45,64,1,10,Penalty,1,825


In [29]:
pass_types = ['Pass Reception', 'Pass Interception Return', 'Pass Incompletion', 'Sack', 'Passing Touchdown', 'Interception Return Touchdown']
rush_types = ['Rush', 'Rushing Touchdown']
punt_types = ['Punt', 'Punt Return Touchdown', 'Blocked Punt', 'Blocked Punt Touchdown']
fg_types = ['Field Goal Good', 'Field Goal Missed', 'Blocked Field Goal']

def getPlayCall(x):
    if x in pass_types:
            return 'pass'
    elif x in rush_types:
        return 'rush'
    elif x in punt_types:
        return 'punt'
    elif x in fg_types:
        return 'fg'
    else:
        return None

In [30]:
df['play_call'] = df['play_type'].apply(getPlayCall)
df.head()

,home,away,offense_score,defense_score,period,clock_minutes,clock_seconds,yards_to_goal,down,distance,play_type,is_home,seconds_remaining,play_call
0,Clemson,Wofford,0,0,1,15,0,65,1,10,Kickoff,1,900,None
1,Clemson,Wofford,0,0,1,15,0,75,1,10,Rush,1,900,rush
2,Clemson,Wofford,0,0,1,14,40,71,2,6,Rush,1,880,rush
3,Clemson,Wofford,0,0,1,14,15,66,3,1,Rush,1,855,rush
4,Clemson,Wofford,0,0,1,13,45,64,1,10,Penalty,1,825,None


In [31]:
df.dropna(subset=['play_call'], inplace=True)
df.head()

,home,away,offense_score,defense_score,period,clock_minutes,clock_seconds,yards_to_goal,down,distance,play_type,is_home,seconds_remaining,play_call
1,Clemson,Wofford,0,0,1,15,0,75,1,10,Rush,1,900,rush
2,Clemson,Wofford,0,0,1,14,40,71,2,6,Rush,1,880,rush
3,Clemson,Wofford,0,0,1,14,15,66,3,1,Rush,1,855,rush
5,Clemson,Wofford,0,0,1,13,40,49,1,10,Pass Reception,1,820,pass
6,Clemson,Wofford,0,0,1,13,15,42,2,3,Pass Reception,1,795,pass


In [33]:
plays = df[['offense_score', 'defense_score', 'period', 'yards_to_goal', 'down', 'distance', 'is_home', 'seconds_remaining', 'play_call']]
plays.head()

,offense_score,defense_score,period,yards_to_goal,down,distance,is_home,seconds_remaining,play_call
1,0,0,1,75,1,10,1,900,rush
2,0,0,1,71,2,6,1,880,rush
3,0,0,1,66,3,1,1,855,rush
5,0,0,1,49,1,10,1,820,pass
6,0,0,1,42,2,3,1,795,pass


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [35]:
# split the data set between our independent variables (i.e. features) and our dependent variable or output
play_calls = plays['play_call']
plays = plays.drop(['play_call'], axis=1)

# split the data into training and validation sets
plays_train, plays_validation, calls_train, calls_validation = train_test_split(plays, play_calls, train_size=0.8, test_size=0.2, random_state=0)
plays_train.head()

,offense_score,defense_score,period,yards_to_goal,down,distance,is_home,seconds_remaining
8860,0,14,1,26,1,10,1,321
13440,10,38,2,18,2,3,1,73
3897,14,7,2,61,1,10,0,441
13175,7,13,2,13,1,10,0,594
15939,0,0,1,77,2,4,0,525


In [36]:
y, y_keys = pd.factorize(calls_train)

In [37]:
classifier = RandomForestClassifier(random_state=0, n_estimators=100)
classifier.fit(plays_train, y)

RandomForestClassifier(random_state=0)

In [38]:
classifier.predict(plays_validation)

array([1, 1, 0, ..., 1, 1, 0])

In [39]:
classifier.predict_proba(plays_validation)[0:10]

array([[0.48, 0.52, 0.  , 0.  ],
       [0.46, 0.54, 0.  , 0.  ],
       [0.81, 0.19, 0.  , 0.  ],
       [0.55, 0.44, 0.  , 0.01],
       [0.33, 0.67, 0.  , 0.  ],
       [0.55, 0.45, 0.  , 0.  ],
       [0.58, 0.4 , 0.  , 0.02],
       [0.83, 0.17, 0.  , 0.  ],
       [0.11, 0.89, 0.  , 0.  ],
       [0.42, 0.58, 0.  , 0.  ]])

In [40]:
predicted_calls = y_keys[classifier.predict(plays_validation)]
predicted_calls

Index(['rush', 'rush', 'pass', 'pass', 'rush', 'pass', 'pass', 'pass', 'rush',
       'rush',
       ...
       'pass', 'rush', 'rush', 'punt', 'pass', 'rush', 'punt', 'rush', 'rush',
       'pass'],
      dtype='object', length=3017)

In [41]:
pd.crosstab(calls_validation, predicted_calls, rownames=['Actual Calls'], colnames=['Predicted Calls'])

Predicted Calls,fg,pass,punt,rush
Actual Calls,,,,
fg,50,4,2,6
pass,9,835,7,529
punt,0,1,220,0
rush,4,511,5,834


In [42]:
list(zip(plays_train, classifier.feature_importances_))

[('offense_score', 0.10308959511353917),
 ('defense_score', 0.11149301050935395),
 ('period', 0.03615831437630828),
 ('yards_to_goal', 0.21332964665642748),
 ('down', 0.19877872721308193),
 ('distance', 0.09461095644182452),
 ('is_home', 0.021693350307109907),
 ('seconds_remaining', 0.22084639938235479)]

In [43]:
plays['seconds_remaining'] = ((4 - plays['period']) * 15 * 60 ) + plays['seconds_remaining']
plays = plays.drop(columns=['is_home', 'period'])

In [44]:
plays_train, plays_validation, calls_train, calls_validation = train_test_split(plays, play_calls, train_size=0.8, test_size=0.2, random_state=0)
y, y_keys = pd.factorize(calls_train)

classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(plays_train, y)

predicted_calls = y_keys[classifier.predict(plays_validation)]

pd.crosstab(calls_validation, predicted_calls, rownames=['Actual Calls'], colnames=['Predicted Calls'])

Predicted Calls,fg,pass,punt,rush
Actual Calls,,,,
fg,51,4,2,5
pass,8,835,6,531
punt,0,0,220,1
rush,4,497,6,847


In [45]:
list(zip(plays_train, classifier.feature_importances_))

[('offense_score', 0.10707562622225643),
 ('defense_score', 0.11703639169527336),
 ('yards_to_goal', 0.22756861484303267),
 ('down', 0.19373986465829504),
 ('distance', 0.08989126073365585),
 ('seconds_remaining', 0.26468824184748657)]

In [46]:
plays['margin'] = plays['offense_score'] - plays['defense_score']
plays = plays.drop(columns=['offense_score', 'defense_score'])

In [47]:
plays_train, plays_validation, calls_train, calls_validation = train_test_split(plays, play_calls, train_size=0.8, test_size=0.2, random_state=0)
y, y_keys = pd.factorize(calls_train)

classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(plays_train, y)

predicted_calls = y_keys[classifier.predict(plays_validation)]

pd.crosstab(calls_validation, predicted_calls, rownames=['Actual Calls'], colnames=['Predicted Calls'])

Predicted Calls,fg,pass,punt,rush
Actual Calls,,,,
fg,52,3,2,5
pass,7,852,7,514
punt,0,0,219,2
rush,3,531,7,813


In [48]:
list(zip(plays_train, classifier.feature_importances_))

[('yards_to_goal', 0.2375109676576776),
 ('down', 0.191483383342118),
 ('distance', 0.09220352862488788),
 ('seconds_remaining', 0.30364923058396803),
 ('margin', 0.17515288979134852)]

In [51]:
def predict_call(yards, down, distance, seconds, margin):
    test_plays = pd.DataFrame({'yards_to_goal': [yards], 'down': [down], 'distance': [distance], 'seconds_remaining': [seconds], 'margin': [margin]})
    return y_keys[classifier.predict(test_plays)][0]

In [55]:
call = predict_call(50, 4, 1, 180, -4)
call

'rush'

In [56]:
call = predict_call(50, 4, 1, 180, 10)
call

'punt'